In [ ]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [4]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir ="/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/tomato/train"
test_dir="/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/tomato/test"

In [5]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*"))
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

10 Classes
10000 Train images
100 Test images


In [6]:

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [7]:

input_shape=(1,128,128,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(128,128),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(128,128),batch_size=32)

Found 10000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [8]:
model = Sequential()
model.add(Conv2D(32, (5, 5),input_shape=input_shape,activation='relu',name="conv2d_1"))
model.add(MaxPooling2D(pool_size=(3, 3),name="max_pooling2d_1"))
model.add(Conv2D(32, (3, 3),activation='relu',name="conv2d_2"))
model.add(MaxPooling2D(pool_size=(2, 2),name="max_pooling2d_2"))
model.add(Conv2D(64, (3, 3),activation='relu',name="conv2d_3"))
model.add(MaxPooling2D(pool_size=(2, 2),name="max_pooling2d_3"))
model.add(Flatten(name="flatten_1"))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 124, 124, 32)      2432      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 41, 41, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 19, 19, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 17, 17, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 8, 8, 64)          0

In [9]:

validation_generator = train_datagen.flow_from_directory(
                       test_dir,
                       target_size=(128,128),
                       batch_size=32)

Found 100 images belonging to 10 classes.


In [13]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
history1 = model.fit(
    train_generator,#egitim verileri
    steps_per_epoch=None,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=None,
    verbose=1,
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)],
    shuffle=True
    )



Epoch 1/5
313/313 [==============================] - 271s 855ms/step - loss: 0.3272 - accuracy: 0.8889 - val_loss: 0.2992 - val_accuracy: 0.9200 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 253s 809ms/step - loss: 0.2816 - accuracy: 0.9009 - val_loss: 0.3476 - val_accuracy: 0.8700 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 243s 775ms/step - loss: 0.2557 - accuracy: 0.9111 - val_loss: 0.2795 - val_accuracy: 0.9100 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 241s 770ms/step - loss: 0.2499 - accuracy: 0.9149 - val_loss: 0.1898 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 241s 770ms/step - loss: 0.2686 - accuracy: 0.9089 - val_loss: 0.2630 - val_accuracy: 0.8900 - lr: 0.0010


In [15]:
model.save('/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/ipynb/model/CNN_model.h5')

In [16]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(validation_generator)
print(f"Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
4/4 [==============================] - 4s 721ms/step - loss: 0.2576 - accuracy: 0.9000
Accuracy: 89.99999761581421
